In [1]:
import sys, os
parent_dir = os.getcwd()
path = os.path.dirname(parent_dir)
sys.path.append(path)

from gym_homer.envs.test_env_v0 import HomerEnv
import pandas as pd
import numpy as np

import gym, torch, numpy as np, torch.nn as nn
from tianshou.utils import WandbLogger
from torch.utils.tensorboard import SummaryWriter
import tianshou as ts
from gym import spaces, wrappers

In [2]:
# Load Data
data = pd.read_csv(path+"/test_env_data.csv", index_col=False).fillna(0)
pd.DataFrame(data)

,time_x,time_y,weekend,month_x,month_y,region_1,region_2,region_3,solar,loads,import_tariff,export_tariff,max_d,max_c,soc
0,0.000,1.000,0,0.0,1.0,0,0,0,0,0.5,0.05,0.0,0.0,0.0,0.0
1,0.500,0.866,0,0.0,1.0,0,0,0,-1,0.5,0.05,0.0,0.0,0.0,0.0
2,0.866,0.500,0,0.0,1.0,0,0,0,-1,0.5,0.05,0.0,0.0,0.0,0.0
3,1.000,0.000,0,0.0,1.0,0,0,0,0,0.5,0.05,0.0,0.0,0.0,0.0
4,0.866,-0.500,0,0.0,1.0,0,0,0,0,0.5,0.05,0.0,0.0,0.0,0.0
5,0.500,-0.866,0,0.0,1.0,0,0,0,0,0.5,0.05,0.0,0.0,0.0,0.0
6,0.000,-1.000,0,0.0,1.0,0,0,0,0,0.5,0.05,0.0,0.0,0.0,0.0
7,-0.500,-0.866,0,0.0,1.0,0,0,0,0,0.5,2.00,2.0,0.0,0.0,0.0
8,-0.866,-0.500,0,0.0,1.0,0,0,0,0,0.5,2.00,2.0,0.0,0.0,0.0
9,-1.000,-0.000,0,0.0,1.0,0,0,0,0,0.5,0.05,0.0,0.0,0.0,0.0


In [3]:
#task = 'HomerEnv-v0'
env = HomerEnv(data=data, start_soc='random')
lr, epoch, batch_size = 1e-3, 20, 64
train_num, test_num = 10, 100
gamma, n_step, target_freq = 0.9, 3, 320
buffer_size = 20000
eps_train, eps_test = 0.1, 0.05
step_per_epoch, step_per_collect = 10000, len(data) * 10
logger = ts.utils.TensorboardLogger(SummaryWriter('log/dqn'))  # TensorBoard is supported!
# For other loggers: https://tianshou.readthedocs.io/en/master/tutorials/logger.html

# you can also try with SubprocVectorEnv
train_envs = ts.env.DummyVectorEnv([lambda: HomerEnv(data=data) for _ in range(train_num)])
test_envs = ts.env.DummyVectorEnv([lambda: HomerEnv(data=data) for _ in range(test_num)])

from tianshou.utils.net.common import Net
# you can define other net by following the API:
# https://tianshou.readthedocs.io/en/master/tutorials/dqn.html#build-the-network
#env = gym.make('gym_homer/HomerEnv-v0', data=data)
wrapped_env = wrappers.FlattenObservation(env)
state_shape = wrapped_env.observation_space.shape or wrapped_env.observation_space.n
action_shape = wrapped_env.action_space.shape or wrapped_env.action_space.n
net = Net(state_shape=state_shape, action_shape=action_shape, hidden_sizes=[64, 128, 8])
optim = torch.optim.Adam(net.parameters(), lr=lr)

policy = ts.policy.DQNPolicy(net, optim, gamma, n_step, target_update_freq=target_freq)
train_collector = ts.data.Collector(policy, train_envs, ts.data.VectorReplayBuffer(buffer_size, train_num), 
                                    exploration_noise=True)
test_collector = ts.data.Collector(policy, test_envs, exploration_noise=True)  # because DQN uses epsilon-greedy method

In [4]:
result = ts.trainer.offpolicy_trainer(
    policy, train_collector, test_collector, epoch, 
    step_per_epoch, step_per_collect,
    test_num, batch_size, update_per_step=1 / step_per_collect,
    train_fn=lambda epoch, env_step: policy.set_eps(eps_train),
    test_fn=lambda epoch, env_step: policy.set_eps(eps_test),
    stop_fn=lambda mean_rewards: mean_rewards >= 2,
    logger=logger)
print(f'Finished training! Use {result["duration"]}')

Epoch #1: 10080it [00:00, 10539.14it/s, env_step=10080, len=11, loss=0.398, n/ep=10, n/st=120, rew=1.36]


Epoch #1: test_reward: 1.720237 ± 0.941230, best_reward: 1.911164 ± 0.338002 in #0


Epoch #2: 10080it [00:00, 11076.98it/s, env_step=20160, len=11, loss=0.257, n/ep=10, n/st=120, rew=1.95]


Epoch #2: test_reward: 1.701639 ± 0.876171, best_reward: 1.911164 ± 0.338002 in #0


Epoch #3: 10080it [00:00, 11489.91it/s, env_step=30240, len=11, loss=0.240, n/ep=10, n/st=120, rew=1.68]


...

Epoch #18: test_reward: 1.806000 ± 0.340480, best_reward: 1.911164 ± 0.338002 in #0


Epoch #19: 10080it [00:00, 11651.09it/s, env_step=191520, len=11, loss=0.158, n/ep=10, n/st=120, rew=1.97]


Epoch #19: test_reward: 1.731440 ± 0.899489, best_reward: 1.911164 ± 0.338002 in #0


Epoch #20: 10080it [00:00, 11541.45it/s, env_step=201600, len=11, loss=0.162, n/ep=10, n/st=120, rew=1.54]


Epoch #20: test_reward: 1.666500 ± 0.938078, best_reward: 1.911164 ± 0.338002 in #0
Finished training! Use 18.84s


In [5]:
policy.eval()
policy.set_eps(20)
collector = ts.data.Collector(policy, env)
collector.collect(n_episode=10)

/Users/brody/school/cap_ve/lib/python3.10/site-packages/tianshou/data/collector.py:68: UserWarning: Single environment detected, wrap to DummyVectorEnv.
  warnings.warn("Single environment detected, wrap to DummyVectorEnv.")


{'n/ep': 10,
 'n/st': 110,
 'rews': array([ 1.95      , -2.175     , -2.16750588, -2.14728586, -2.16642071,
         1.95      , -2.175     ,  1.95      ,  1.95      , -2.1567717 ]),
 'lens': array([11, 11, 11, 11, 11, 11, 11, 11, 11, 11]),
 'idxs': array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0]),
 'rew': -0.5187984157819301,
 'len': 11.0,
 'rew_std': 2.0157800775477988,
 'len_std': 0.0}

## Continuous action space

In [6]:
#task = 'HomerEnv-v1'
env = HomerEnv(data=data, start_soc='random', discrete=False)
lr, epoch, batch_size = 1e-3, 20, 64
train_num, test_num = 10, 100
gamma, n_step, target_freq = 0.9, 3, 320
buffer_size = 20000
eps_train, eps_test = 0.1, 0.05
step_per_epoch, step_per_collect = 10000, len(data) * 10
logger = ts.utils.TensorboardLogger(SummaryWriter('log/dqn'))  # TensorBoard is supported!
# For other loggers: https://tianshou.readthedocs.io/en/master/tutorials/logger.html

# you can also try with SubprocVectorEnv
train_envs = ts.env.DummyVectorEnv([lambda: HomerEnv(data=data) for _ in range(train_num)])
test_envs = ts.env.DummyVectorEnv([lambda: HomerEnv(data=data) for _ in range(test_num)])

from tianshou.utils.net.common import Net
# you can define other net by following the API:
# https://tianshou.readthedocs.io/en/master/tutorials/dqn.html#build-the-network
#env = gym.make('gym_homer/HomerEnv-v0', data=data)
wrapped_env = wrappers.FlattenObservation(env)
state_shape = wrapped_env.observation_space.shape or wrapped_env.observation_space.n
action_shape = wrapped_env.action_space.shape or wrapped_env.action_space.n
net = Net(state_shape=state_shape, action_shape=action_shape, hidden_sizes=[64, 128, 8])
optim = torch.optim.Adam(net.parameters(), lr=lr)

policy = ts.policy.DQNPolicy(net, optim, gamma, n_step, target_update_freq=target_freq)
train_collector = ts.data.Collector(policy, train_envs, ts.data.VectorReplayBuffer(buffer_size, train_num), 
                                    exploration_noise=True)
test_collector = ts.data.Collector(policy, test_envs, exploration_noise=True)  # because DQN uses epsilon-greedy method

In [7]:
result = ts.trainer.offpolicy_trainer(
    policy, train_collector, test_collector, epoch, 
    step_per_epoch, step_per_collect,
    test_num, batch_size, update_per_step=1 / step_per_collect,
    train_fn=lambda epoch, env_step: policy.set_eps(eps_train),
    test_fn=lambda epoch, env_step: policy.set_eps(eps_test),
    stop_fn=lambda mean_rewards: mean_rewards >= 2,
    logger=logger)
print(f'Finished training! Use {result["duration"]}')

Epoch #1: 10080it [00:00, 12271.38it/s, env_step=10080, len=11, loss=0.451, n/ep=10, n/st=120, rew=-2.18]


Epoch #1: test_reward: -2.175000 ± 0.000000, best_reward: -2.175000 ± 0.000000 in #0


Epoch #2: 10080it [00:00, 10354.89it/s, env_step=20160, len=11, loss=0.049, n/ep=10, n/st=120, rew=-2.18]


Epoch #2: test_reward: -2.175000 ± 0.000000, best_reward: -2.175000 ± 0.000000 in #0


Epoch #3: 10080it [00:00, 11182.02it/s, env_step=30240, len=11, loss=0.002, n/ep=10, n/st=120, rew=-2.18]


...

Epoch #18: test_reward: -2.175000 ± 0.000000, best_reward: -2.175000 ± 0.000000 in #0


Epoch #19: 10080it [00:00, 12331.58it/s, env_step=191520, len=11, loss=0.000, n/ep=10, n/st=120, rew=-2.18]


Epoch #19: test_reward: -2.175000 ± 0.000000, best_reward: -2.175000 ± 0.000000 in #0


Epoch #20: 10080it [00:00, 12382.71it/s, env_step=201600, len=11, loss=0.000, n/ep=10, n/st=120, rew=-2.18]


Epoch #20: test_reward: -2.175000 ± 0.000000, best_reward: -2.175000 ± 0.000000 in #0
Finished training! Use 18.29s


In [8]:
policy.eval()
policy.set_eps(20)
collector = ts.data.Collector(policy, env)
collector.collect(n_episode=10)

{'n/ep': 10,
 'n/st': 110,
 'rews': array([-2.175, -2.175, -2.175, -2.175, -2.175, -2.175, -2.175, -2.175,
        -2.175, -2.175]),
 'lens': array([11, 11, 11, 11, 11, 11, 11, 11, 11, 11]),
 'idxs': array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0]),
 'rew': -2.175000002607703,
 'len': 11.0,
 'rew_std': 0.0,
 'len_std': 0.0}